In [2]:
import pandas as pd

In [3]:
cc_inv = pd.read_excel('Courier Company - Invoice.xlsx')
X_order_rep = pd.read_excel("Company X - Order Report.xlsx")
X_pcz = pd.read_excel('Company X - Pincode Zones.xlsx')
X_sku = pd.read_excel('Company X - SKU Master.xlsx')
cc_rate = pd.read_excel('Courier Company - Rates.xlsx')

In [4]:
# function to make slabs for weight 
def half_round_up(x):
    x = str(x)
    pos = x.find('.')
    dec = x[pos:]
    whl = x[:pos]
    dec, whl = float(dec), int(whl)
    dec = round(dec, 1)
    if dec <= 0.5:
        dec = .5
        res_num = whl+dec
    else:
        dec = 0
        res_num = whl+1
    return res_num


In [5]:
# function to calculate weight usning order number and Company X's order Report
def weight_cal(orderID):
    find = X_order_rep[X_order_rep['ExternOrderNo'] == orderID]
    items = []
    for value in find['SKU']:
            weight = (X_sku.loc[X_sku['SKU'] == value, 'Weight (g)'].tolist()[0] *  find.loc[find['SKU'] == value, 'Order Qty'].tolist()[0])/1000
            items.append(weight)
    total = sum(items)
    return total


In [6]:
# function to calculate price on 4 different scenarios fwd fixed, fwd additional, fwd + rto fixed, fwd + rto additonal and different zones
def price_cal(orderID):
    if cc_inv.loc[cc_inv['Order ID']==orderID , 'Type of Shipment'].tolist()[0] == 'Forward charges':
        charge_f = 'fwd_'
    else:
         charge_f = 'rto_'
            
    weight = res_df.loc[res_df['Order ID']==orderID, 'Weight slab as per X (KG)'].tolist()[0]
    if weight <= 0.5:
        charge_e = '_fixed'
    else:
        charge_e = '_additional'
    zone = res_df.loc[res_df['Order ID']==orderID, 'Delivery Delivery Zone charged by Courier Company as per X'].tolist()[0]
    charge = charge_f + zone + charge_e
    cost = None
    if charge_f == 'fwd_' and charge_e == '_fixed':
        cost = cc_rate[charge].item()
    elif charge_f == 'fwd_' and charge_e == '_additional':
        cost = (weight/.5 - 1) * cc_rate[charge].item() + cc_rate[charge_f + zone + '_fixed'].item()
    elif charge_f == 'rto_' and charge_e == '_fixed':
        cost = cc_rate[charge].item() + cc_rate['fwd_'+zone+charge_e].item()
    elif charge_f == 'rto_' and charge_e == '_additional':
        cost = (weight/.5 -1)*cc_rate[charge].item() + (weight/.5 - 1) * cc_rate['fwd_' + zone + charge_e].item() + cc_rate[charge_f + zone + '_fixed'].item() + cc_rate['fwd_' + zone + '_fixed'].item()
    return cost


In [7]:
#function to get data from courier company's invoice sheet based on order ID to avoid any error
def get_data(tabFrom, tabTo ,val, fetchVal, colName):
    list1 = []
    for value in tabTo[val]:
        item = tabFrom.loc[tabFrom[val] == value, fetchVal].item()
        list1.append(item)
    tabTo[fetchVal] = list1
    tabTo.columns = tabTo.columns.str.replace(fetchVal, colName)
    return tabTo

In [8]:
res_df = pd.DataFrame(cc_inv['Order ID'])

In [9]:
get_data(cc_inv, res_df, 'Order ID', 'AWB Code', 'AWB Number')

,Order ID,AWB Number
0,2001806232,1091117222124
1,2001806273,1091117222194
2,2001806408,1091117222931
3,2001806458,1091117223244
4,2001807012,1091117229345
...,...,...
119,2001812941,1091118551656
120,2001809383,1091117614452
121,2001820978,1091120922803
122,2001811475,1091121844806


In [10]:
res_df['Total weight as per X (KG)'] = res_df.apply(lambda row:weight_cal(row['Order ID']), axis=1)

In [11]:
res_df['Weight slab as per X (KG)'] = res_df.apply(lambda row:half_round_up(weight_cal(row['Order ID'])), axis=1)

In [12]:
get_data(cc_inv, res_df, 'Order ID', 'Charged Weight', 'Total weight as per Courier Company (KG)')

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG)
0,2001806232,1091117222124,1.302,1.5,1.30
1,2001806273,1091117222194,0.615,1.0,1.00
2,2001806408,1091117222931,2.265,2.5,2.50
3,2001806458,1091117223244,0.700,1.0,1.00
4,2001807012,1091117229345,0.240,0.5,0.15
...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73
120,2001809383,1091117614452,0.607,1.0,0.50
121,2001820978,1091120922803,0.515,0.5,0.50
122,2001811475,1091121844806,0.689,1.0,0.50


In [13]:
res_df['Weight slab charged by Courier Company (KG)'] = res_df.apply(lambda row:half_round_up(row['Total weight as per Courier Company (KG)']), axis=1)

In [14]:
res_df['Delivery Zone as per X'] = [X_pcz[(X_pcz['Warehouse Pincode']==cc_inv['Warehouse Pincode'][i]) & (X_pcz['Customer Pincode']==cc_inv['Customer Pincode'][i])]['Zone'].tolist()[0] for i in range(len(cc_inv.index))]

In [15]:
get_data(cc_inv, res_df, 'Order ID', 'Zone', 'Delivery Zone charged by Courier Company')

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Delivery Zone charged by Courier Company as per X,Delivery Zone charged by Courier Company
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d
1,2001806273,1091117222194,0.615,1.0,1.00,1.5,d,d
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d
3,2001806458,1091117223244,0.700,1.0,1.00,1.5,b,b
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d
...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d
120,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d
121,2001820978,1091120922803,0.515,0.5,0.50,0.5,b,d
122,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b


In [16]:
res_df['Expected Charge as per X (Rs.)'] = res_df.apply(lambda row:price_cal(row['Order ID']), axis=1)

In [17]:
get_data(cc_inv, res_df, 'Order ID', 'Billing Amount (Rs.)', 'Charges Billed by Courier Company (Rs.)')

C:\Users\91703\AppData\Local\Temp/ipykernel_13424/1565350463.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tabTo.columns = tabTo.columns.str.replace(fetchVal, colName)


,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Delivery Zone charged by Courier Company as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Billing Amount (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.5,d,d,90.2,90.2
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6
3,2001806458,1091117223244,0.700,1.0,1.00,1.5,b,b,61.3,61.3
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4
...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d,33.0,90.2
120,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d,110.1,86.7
121,2001820978,1091120922803,0.515,0.5,0.50,0.5,b,d,33.0,45.4
122,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0


In [20]:
res_df['Difference Between Expected Charges and Billed Charges (Rs.)'] = res_df.apply(lambda row: round(row['Expected Charge as per X (Rs.)'] - row['Billing Amount (Rs.)'], 3), axis = 1)

In [21]:
res_df.columns = res_df.columns.str.replace('Billing Amount (Rs.)', 'Charges Billed by Courier Company (Rs.)')


C:\Users\91703\AppData\Local\Temp/ipykernel_13424/3548730589.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  res_df.columns = res_df.columns.str.replace('Billing Amount (Rs.)', 'Charges Billed by Courier Company (Rs.)')


In [22]:
res_df

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Delivery Zone charged by Courier Company as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Billing Amount (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.5,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.5,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d,33.0,90.2,-57.2
120,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d,110.1,86.7,23.4
121,2001820978,1091120922803,0.515,0.5,0.50,0.5,b,d,33.0,45.4,-12.4
122,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1


In [23]:
res_df.rename(columns={'Billing Amount (Rs.)': 'Charges Billed by Courier Company (Rs.)'}, inplace=True)

In [24]:
res_df

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Delivery Zone charged by Courier Company as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.5,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.5,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d,33.0,90.2,-57.2
120,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d,110.1,86.7,23.4
121,2001820978,1091120922803,0.515,0.5,0.50,0.5,b,d,33.0,45.4,-12.4
122,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1


In [25]:
res_df.to_excel('result_data.xlsx')